# Create bronze tables

This code is designed for Databricks to process and ingest various JSON datasets from an S3 bucket into a Delta Lake catalog. It defines custom schemas using PySpark's StructType, MapType, and ArrayType to handle complex, nested JSON structures. The script iterates over the S3 bucket's contents, identifying directories that start with specific prefixes like sysmon, wineventlogs, or zeek. For each identified directory, it reads the corresponding JSON files into DataFrames using the predefined schemas and writes them to Delta tables in the specified catalog and schema. The get_zeek_sourcetypes function is utilized to dynamically discover and process Zeek log files based on their filenames. This approach ensures that structured and semi-structured log data is efficiently ingested into Databricks for further analysis. Lastly, each DataFrame is written as Delta table in Unity Catalog under the `btv_dc30.bronze` schema, overwriting any existing tables with the same name. Each folder becomes a table, with its name used as the table name.

In [ ]:
__CATALOG = "btv_dc30"
__SCHEMA = "bronze"
__BUCKET = "btv-dc30-bronze-t7rrh"

In [ ]:
from pyspark.sql.functions import to_timestamp, from_unixtime, col, timestamp_millis
from pyspark.sql.types import (
    StructType, 
    StructField, 
    StringType, 
    MapType, 
    ArrayType, 
    BooleanType, 
    IntegerType,
    TimestampType
)


sysmon_bronze_schema = StructType([
    StructField("agent", MapType(StringType(), StringType()), True),
    StructField("@timestamp", TimestampType(), True),
    StructField("rule", MapType(StringType(), StringType()), True),
    StructField("winlog", 
        StructType([
            StructField("api", StringType(), True),
            StructField("channel", StringType(), True),
            StructField("computer_name", StringType(), True),
            StructField("event_data", MapType(StringType(), StringType()), True),
            StructField("event_id", StringType(), True),
            StructField("keywords", ArrayType(StringType()), True),
            StructField("opcode", StringType(), True),
            StructField("process", MapType(StringType(), IntegerType()), True),
            StructField("provider_guid", StringType(), True),
            StructField("provider_name", StringType(), True),
            StructField("task", StringType(), True),
            StructField("record_id", StringType(), True),
            StructField("version", StringType(), True),
        ]),
        True,
    ),
    StructField("@version", StringType(), True),
    StructField("host", MapType(StringType(), StringType()), True),
    StructField("message", StringType(), True),
    StructField("process", MapType(StringType(), StringType()), True),
    StructField("ecs", MapType(StringType(), StringType()), True),
    StructField("event", MapType(StringType(), StringType()), True),
    StructField("tags", ArrayType(StringType()), True),
    StructField("log", MapType(StringType(), StringType()), True),
    StructField("file", MapType(StringType(), StringType()), True),
    StructField("registry", MapType(StringType(), StringType()), True)
])

wineventlogs_bronze_schema = StructType([
    StructField("tags", ArrayType(StringType()), True),
    StructField("winlog", MapType(StringType(), StringType()), True),
    StructField("computer_name", StringType(), True),
    StructField("event_id", StringType(), True),
    StructField("api", StringType(), True),
    StructField("channel", StringType(), True),
    StructField("provider_guid", StringType(), True),
    StructField("opcode", StringType(), True),
    StructField("task", StringType(), True),
    StructField("record_id", IntegerType(), True),
    StructField("version", IntegerType(), True),
    StructField("process", MapType(StringType(), StringType()), True),
    StructField("keywords", ArrayType(StringType()), True),
    StructField("provider_name", StringType(), True),
    StructField("log", MapType(StringType(), StringType()), True),
    StructField("@timestamp", TimestampType(), True),
    StructField("agent", MapType(StringType(), StringType()), True),
    StructField("message", StringType(), True),
    StructField("@version", StringType(), True),
    StructField("event", MapType(StringType(), StringType()), True),
    StructField("host", MapType(StringType(), StringType()), True),
    StructField("ecs", MapType(StringType(), StringType()), True)
])

def get_dir_content(ls_path):
    dir_paths = dbutils.fs.ls(ls_path)
    subdir_paths = [get_dir_content(p.path) for p in dir_paths if p.isDir() and p.path != ls_path]
    flat_subdir_paths = [p for subdir in subdir_paths for p in subdir]
    return list(map(lambda p: p.path, dir_paths)) + flat_subdir_paths

def get_zeek_sourcetypes(ls_path):
    sourcetypes = set()
    paths = get_dir_content(ls_path)
    for p in paths:
        if not p.endswith(".log"):
            continue
        sourcetype = p.split("/")[-1].split(".", 1)[0]
        sourcetypes.add(sourcetype)

    return sourcetypes

for datasource in dbutils.fs.ls(f"s3://{__BUCKET}/"):
    print(datasource.name.rstrip("/"))
    if datasource.name.startswith("_"):
        continue
    elif datasource.name.startswith("sysmon"):
        print (f"{__CATALOG}.{__SCHEMA}.{datasource.name.rstrip('/')}")
        df = spark.read.schema(sysmon_bronze_schema).json(f"s3://{__BUCKET}/{datasource.name.rstrip('/')}")
        df.write.mode("overwrite").saveAsTable(f"`{__CATALOG}`.{__SCHEMA}.{datasource.name.rstrip('/')}")
    elif datasource.name.startswith("wineventlogs"):
        print (f"{__CATALOG}.{__SCHEMA}.{datasource.name.rstrip('/')}")
        df = spark.read.schema(wineventlogs_bronze_schema).json(f"s3://{__BUCKET}/{datasource.name.rstrip('/')}")
        df.write.mode("overwrite").saveAsTable(f"`{__CATALOG}`.{__SCHEMA}.{datasource.name.rstrip('/')}")
    # Zeek
    elif datasource.name.startswith("zeek"):
        sourcetypes = get_zeek_sourcetypes(f"s3://{__BUCKET}/zeek/")
        for sourcetype in sourcetypes:
            if sourcetype == "conn-summary":
                continue
            print (f"{__CATALOG}.{__SCHEMA}.{datasource.name.rstrip('/')}_{sourcetype}")
            df = spark.read.json(f"s3://{__BUCKET}/{datasource.name.rstrip('/')}/**/{sourcetype}*")
            df_with_timestamp = df.withColumn("ts", to_timestamp(from_unixtime(col("ts"))))
            wp = f"{__CATALOG}.{__SCHEMA}.{datasource.name.rstrip('/')}_{sourcetype.replace('-', '_')}"
            df_with_timestamp.write.mode("overwrite").saveAsTable(wp)
    else:
        print (f"{__CATALOG}.{__SCHEMA}.{datasource.name.rstrip('/')}")
        df = spark.read.json(f"s3://{__BUCKET}/{datasource.name.rstrip('/')}")
        df_with_timestamp = df.withColumn("@timestamp", to_timestamp("@timestamp", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
        df_with_timestamp.write.mode("overwrite") \
                .saveAsTable(f"`{__CATALOG}`.{__SCHEMA}.{datasource.name.rstrip('/')}")
        

hmail
btv_dc30.bronze.hmail
osquery
btv_dc30.bronze.osquery
sysmon
btv_dc30.bronze.sysmon
wineventlogs
btv_dc30.bronze.wineventlogs
zeek
btv_dc30.bronze.zeek_kerberos
btv_dc30.bronze.zeek_conn
btv_dc30.bronze.zeek_cluster
btv_dc30.bronze.zeek_capture_loss
btv_dc30.bronze.zeek_notice
btv_dc30.bronze.zeek_dns
btv_dc30.bronze.zeek_pe
btv_dc30.bronze.zeek_known_services
btv_dc30.bronze.zeek_smb_files
btv_dc30.bronze.zeek_reporter
btv_dc30.bronze.zeek_broker
btv_dc30.bronze.zeek_http
btv_dc30.bronze.zeek_smtp
btv_dc30.bronze.zeek_ntp
btv_dc30.bronze.zeek_smb_mapping
btv_dc30.bronze.zeek_dce_rpc
btv_dc30.bronze.zeek_tunnel
btv_dc30.bronze.zeek_weird
btv_dc30.bronze.zeek_rdp
btv_dc30.bronze.zeek_dpd
btv_dc30.bronze.zeek_x509
btv_dc30.bronze.zeek_ssl
btv_dc30.bronze.zeek_known_certs
btv_dc30.bronze.zeek_files
btv_dc30.bronze.zeek_stats
btv_dc30.bronze.zeek_ntlm
btv_dc30.bronze.zeek_software


# Verify table creation


In [ ]:
%sql
SELECT * FROM btv_dc30.bronze.hmail
LIMIT 3

@timestamp,@version,agent,ecs,fields,host,input,log,message,service,tags
2022-02-12T18:55:28.706Z,1,"List(311bb01e-323f-46fb-8eea-3ddbabb8d60b, files, 6c9a9834-52f8-4d57-8c16-4f3bd9022475, files, filebeat, 7.16.2)",List(1.12.0),List(List(hmail)),List(files),List(log),"List(List(C:\Program Files (x86)\hMailServer\Logs\hmailserver_2022-02-12.log), 760962)","""TCPIP"" 2784 ""2022-02-12 18:55:26.773"" ""TCP - 172.16.50.130 connected to 172.16.50.110:143.""",List(hmail),List(beats_input_codec_plain_applied)
2022-02-12T18:55:28.706Z,1,"List(311bb01e-323f-46fb-8eea-3ddbabb8d60b, files, 6c9a9834-52f8-4d57-8c16-4f3bd9022475, files, filebeat, 7.16.2)",List(1.12.0),List(List(hmail)),List(files),List(log),"List(List(C:\Program Files (x86)\hMailServer\Logs\hmailserver_2022-02-12.log), 761056)","""TCPIP"" 2796 ""2022-02-12 18:55:26.773"" ""TCP - 172.16.50.130 connected to 172.16.50.110:143.""",List(hmail),List(beats_input_codec_plain_applied)
2022-02-12T18:55:28.707Z,1,"List(311bb01e-323f-46fb-8eea-3ddbabb8d60b, files, 6c9a9834-52f8-4d57-8c16-4f3bd9022475, files, filebeat, 7.16.2)",List(1.12.0),List(List(hmail)),List(files),List(log),"List(List(C:\Program Files (x86)\hMailServer\Logs\hmailserver_2022-02-12.log), 761150)","""IMAPD"" 2784 3171 ""2022-02-12 18:55:26.773"" ""172.16.50.130"" ""SENT: * OK IMAPrev1""",List(hmail),List(beats_input_codec_plain_applied)


In [ ]:
%sql
SELECT * FROM btv_dc30.bronze.osquery
LIMIT 3

@timestamp,@version,agent,ecs,event,fileset,host,input,json,log,service,tags
2022-02-12T18:21:16.095Z,1,"List(1f678581-8e27-4bf9-9591-5b13d5514410, ip-172-16-55-120, 951fc96d-4fb6-42ee-b824-ef88c3ff3b03, ip-172-16-55-120, filebeat, 7.17.0)",List(1.12.0),"List(osquery.result, osquery)",List(result),List(ip-172-16-55-120),List(log),"List(removed, Sat Feb 12 18:21:07 2022 UTC, List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 0, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 0, 0, null, null, null, 24082345, null, null, null, null, null, null, null, 3506, 0, null, null, null, null, null, null, null, null, null, null, null, null, 75793, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 697, null, null, null, null, null, 67173, null, null, null, null, 33177, null, null, null, null, null, null, null, null, null, 26696, null, null, null, null, null, null), 6780, 0, ip-172-16-55-120, cpu_time, false, 1644690067)","List(List(/var/log/osquery/osqueryd.results.log), 88307433)",List(osquery),List(beats_input_raw_event)
2022-02-12T18:21:16.095Z,1,"List(1f678581-8e27-4bf9-9591-5b13d5514410, ip-172-16-55-120, 951fc96d-4fb6-42ee-b824-ef88c3ff3b03, ip-172-16-55-120, filebeat, 7.17.0)",List(1.12.0),"List(osquery.result, osquery)",List(result),List(ip-172-16-55-120),List(log),"List(added, Sat Feb 12 18:21:07 2022 UTC, List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 0, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 0, 0, null, null, null, 24088390, null, null, null, null, null, null, null, 3506, 0, null, null, null, null, null, null, null, null, null, null, null, null, 75817, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 697, null, null, null, null, null, 67195, null, null, null, null, 33187, null, null, null, null, null, null, null, null, null, 26699, null, null, null, null, null, null), 6780, 0, ip-172-16-55-120, cpu_time, false, 1644690067)","List(List(/var/log/osquery/osqueryd.results.log), 88308153)",List(osquery),List(beats_input_raw_event)
2022-02-12T18:21:16.095Z,1,"List(1f678581-8e27-4bf9-9591-5b13d5514410, ip-172-16-55-120, 951fc96d-4fb6-42ee-b824-ef88c3ff3b03, ip-172-16-55-120, filebeat, 7.17.0)",List(1.12.0),"List(osquery.result, osquery)",List(result),List(ip-172-16-55-120),List(log),"List(added, Sat Feb 12 18:21:13 2022 UTC, List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 2022-02-12T18:21:09.180579+00:00, null, null, null, null, null, null, null, null, null, null, null, daemon, null, null, null, null, null, null, null, null, null, ip-172-16-55-120, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 2022-02-12T18:21:09.180Z#011INFO#011[monitoring]#011log/log.go:184#011Non-zero metrics in the last 30s#011{""monitoring"": {""metrics"": {""beat"":{""cgroup"":{""cpuacct"":{""total"":{""ns"":9658185}}},""cpu"":{""system"":{""ticks"":22820,""time"":{""ms"":7}},""total"":{""ticks"":177040,""time"":{""ms"":8},""value"":177040},""user"":{""ticks"":154220,""time"":{""ms"":1}}},""handles"":{""limit"":{""hard"":524288,""soft"":1024},""open"":12},""info"":{""ephemeral_id"":""1f678581-8e27-4bf9-9591-5b13d5514410"",""uptime"":{""ms"":242820612},""version"":""7.17.0""},""memstats"":{""gc_next"":21543440,""memory_alloc"":15042776,""memory_total"":12735397576,""rss"":100274176},""runtime"":{""goroutines"":36}},""filebeat"":{""events"":{""added"":7,""done"":7},""harvester"":{""open

In [ ]:
%sql
SELECT * FROM btv_dc30.bronze.sysmon
LIMIT 3

agent,@timestamp,rule,winlog,@version,host,message,process,ecs,event,tags,log,file
"Map(name -> wkst08, hostname -> wkst08, version -> 7.16.2, id -> ede5221c-9b85-4b9c-a9df-65d46bd0af04, ephemeral_id -> 659fbb60-a4d9-4ebf-8f6e-b7c4e87574a2, type -> winlogbeat)",2022-02-19T19:34:19.807Z,"Map(name -> technique_id=T1047,technique_name=File System Permissions Weakness)","List(wineventlog, Microsoft-Windows-Sysmon/Operational, wkst08.magnumtempus.financial, Map(User -> NT AUTHORITY\LOCAL SERVICE, CreationUtcTime -> 2022-02-12 01:44:13.404), 11, null, Info, null, {5770385F-C22A-43E0-BF4C-06F5698FFBD9}, Microsoft-Windows-Sysmon, File created (rule: FileCreate), 38314, 2)",1,"Map(mac -> [""06:60:a2:3f:3d:7e"",""00:00:00:00:00:00:00:e0""], name -> wkst08.magnumtempus.financial, ip -> [""fe80::193b:92d2:20fa:85c8"",""172.16.50.137""], hostname -> wkst08, os -> {""platform"":""windows"",""type"":""windows"",""kernel"":""10.0.14393.4770 (rs1_release.211101-1440)"",""family"":""windows"",""version"":""10.0"",""name"":""Windows Server 2016 Datacenter"",""build"":""14393.4770""}, architecture -> x86_64, id -> e0333fc0-5cb7-4293-b6ac-6c3695286056)","File created: RuleName: technique_id=T1047,technique_name=File System Permissions Weakness UtcTime: 2022-02-19 19:34:19.807 ProcessGuid: {E0333FC0-21A7-6211-1300-000000001102} ProcessId: 68 Image: C:\Windows\System32\svchost.exe TargetFilename: C:\Windows\ServiceProfiles\LocalService\AppData\Local\lastalive0.dat CreationUtcTime: 2022-02-12 01:44:13.404 User: NT AUTHORITY\LOCAL SERVICE","Map(executable -> C:\Windows\System32\svchost.exe, entity_id -> {E0333FC0-21A7-6211-1300-000000001102}, pid -> 68, name -> svchost.exe)",Map(version -> 1.12.0),"Map(module -> sysmon, provider -> Microsoft-Windows-Sysmon, code -> 11, category -> [""file""], kind -> event, type -> [""creation""], action -> File created (rule: FileCreate), created -> 2022-02-19T19:34:21.604Z)","List(sysmon, beats_input_codec_plain_applied)",Map(level -> information),"Map(extension -> dat, path -> C:\Windows\ServiceProfiles\LocalService\AppData\Local\lastalive0.dat, name -> lastalive0.dat, directory -> C:\Windows\ServiceProfiles\LocalService\AppData\Local)"
"Map(name -> wkst02, hostname -> wkst02, version -> 7.16.2, id -> ede5221c-9b85-4b9c-a9df-65d46bd0af04, ephemeral_id -> eb859ae7-04cd-41e6-8417-aa8a46e9190f, type -> winlogbeat)",2022-02-19T19:34:24.248Z,"Map(name -> technique_id=T1047,technique_name=File System Permissions Weakness)","List(wineventlog, Microsoft-Windows-Sysmon/Operational, wkst02.magnumtempus.financial, Map(User -> NT AUTHORITY\LOCAL SERVICE, CreationUtcTime -> 2022-02-19 16:10:19.454), 11, null, Info, null, {5770385F-C22A-43E0-BF4C-06F5698FFBD9}, Microsoft-Windows-Sysmon, File created (rule: FileCreate), 39813, 2)",1,"Map(mac -> [""06:94:d6:f4:99:d2"",""00:00:00:00:00:00:00:e0""], name -> wkst02.magnumtempus.financial, ip -> [""fe80::cd03:a025:9781:bb37"",""172.16.50.131"",""fe80::5efe:ac10:3283""], hostname -> wkst02, os -> {""platform"":""windows"",""type"":""windows"",""kernel"":""10.0.14393.4770 (rs1_release.211101-1440)"",""family"":""windows"",""version"":""10.0"",""name"":""Windows Server 2016 Datacenter"",""build"":""14393.4770""}, architecture -> x86_64, id -> d4a1b8f4-baae-4212-b9dd-99aa0716c3e3)","File created: RuleName: technique_id=T1047,technique_name=File System Permissions Weakness UtcTime: 2022-02-19 19:34:24.248 ProcessGuid: {D4A1B8F4-162D-6211-1100-000000001302} ProcessId: 912 Image: C:\Windows\System32\svchost.exe TargetFilename: C:\Windows\ServiceProfiles\LocalService\AppData\Local\lastalive1.dat CreationUtcTime: 2022-02-19 16:10:19.454 User: NT AUTHORITY\LOCAL SERVICE","Map(entity_id -> {D4A1B8F4-162D-6211-1100-000000001302}, executable -> C:\Windows\System32\svchost.exe, pid -> 912, name -> svchost.exe)",Map(version -> 1.12.0),"Map(module -> sysmon, provider -> Microsoft-Windows-Sysmon, code -> 11, category -> [""file""], kind -> event, type -> [""creation""], action -> File created (rule: F

In [ ]:
%sql
SELECT * FROM btv_dc30.bronze.wineventlogs
LIMIT 3

tags,winlog,computer_name,event_id,api,channel,provider_guid,opcode,task,record_id,version,process,keywords,provider_name,log,@timestamp,agent,message,@version,event,host,ecs
List(beats_input_codec_plain_applied),"Map(record_id -> 108612, channel -> System, event_data -> {""UserSid"":""S-1-5-21-2370586174-1517003462-1142029260-1136"",""TSId"":""3""}, process -> {""pid"":4184,""thread"":{""id"":3272}}, computer_name -> wkst06.magnumtempus.financial, provider_name -> Microsoft-Windows-Winlogon, provider_guid -> {DBE9B383-7CF3-4331-91CC-A3CB16A3B538}, opcode -> Info, api -> wineventlog, event_id -> 7002, user -> {""domain"":""NT AUTHORITY"",""name"":""SYSTEM"",""type"":""User"",""identifier"":""S-1-5-18""})",null,null,null,null,null,null,null,null,null,null,null,null,Map(level -> information),2022-02-12T20:14:56.212Z,"Map(name -> wkst06, hostname -> wkst06, version -> 7.16.2, id -> ede5221c-9b85-4b9c-a9df-65d46bd0af04, ephemeral_id -> 7203285e-85aa-46f8-b9f1-ca88ef568144, type -> winlogbeat)",User Logoff Notification for Customer Experience Improvement Program,1,"Map(provider -> Microsoft-Windows-Winlogon, kind -> event, created -> 2022-02-12T20:15:29.248Z, code -> 7002)","Map(mac -> [""06:a3:be:71:d7:f0"",""00:00:00:00:00:00:00:e0""], name -> wkst06.magnumtempus.financial, ip -> [""fe80::a165:e292:4340:72a7"",""172.16.50.135"",""fe80::5efe:ac10:3287""], hostname -> wkst06, os -> {""type"":""windows"",""platform"":""windows"",""family"":""windows"",""kernel"":""10.0.14393.4770 (rs1_release.211101-1440)"",""version"":""10.0"",""name"":""Windows Server 2016 Datacenter"",""build"":""14393.4770""}, architecture -> x86_64, id -> 18e6654d-d024-4c71-ba09-cf96cd5e4a5a)",Map(version -> 1.12.0)
List(beats_input_codec_plain_applied),"Map(record_id -> 108614, channel -> System, event_data -> {""Binary"":""740069006C00650064006100740061006D006F00640065006C007300760063002F0031000000"",""param1"":""Tile Data model server"",""param2"":""stopped""}, process -> {""pid"":556,""thread"":{""id"":1900}}, computer_name -> wkst06.magnumtempus.financial, provider_name -> Service Control Manager, provider_guid -> {555908d1-a6d7-4695-8e1e-26931d2012f4}, keywords -> [""Classic""], api -> wineventlog, event_id -> 7036)",null,null,null,null,null,null,null,null,null,null,null,null,Map(level -> information),2022-02-12T20:14:56.286Z,"Map(name -> wkst06, hostname -> wkst06, version -> 7.16.2, id -> ede5221c-9b85-4b9c-a9df-65d46bd0af04, ephemeral_id -> 7203285e-85aa-46f8-b9f1-ca88ef568144, type -> winlogbeat)",The Tile Data model server service entered the stopped state.,1,"Map(provider -> Service Control Manager, kind -> event, created -> 2022-02-12T20:15:29.249Z, code -> 7036)","Map(mac -> [""06:a3:be:71:d7:f0"",""00:00:00:00:00:00:00:e0""], name -> wkst06.magnumtempus.financial, ip -> [""fe80::a165:e292:4340:72a7"",""172.16.50.135"",""fe80::5efe:ac10:3287""], hostname -> wkst06, os -> {""type"":""windows"",""platform"":""windows"",""family"":""windows"",""kernel"":""10.0.14393.4770 (rs1_release.211101-1440)"",""version"":""10.0"",""name"":""Windows Server 2016 Datacenter"",""build"":""14393.4770""}, architecture -> x86_64, id -> 18e6654d-d024-4c71-ba09-cf96cd5e4a5a)",Map(version -> 1.12.0)
List(beats_input_codec_plain_applied),"Map(record_id -> 108616, channel -> System, event_data -> {""Binary"":""670070007300760063002F0034000000"",""param1"":""Group Policy Client"",""param2"":""running""}, process -> {""thread"":{""id"":1900},""pid"":556}, computer_name -> wkst06.magnumtempus.financial, provider_name -> Service Control Manager, provider_guid -> {555908d1-a6d7-4695-8e1e-26931d2012f4}, keywords -> [""Classic""], api -> wineventlog, event_id -> 7036)",null,null,null,null,null,null,null,null,null,null,null,null,Map(level -> information),2022-02-12T20:14:56.318Z,"Map(name -> wkst06, hostname -> wkst06, version -> 7.16.2, id -> ede5221c-9b85-4b9c-a9df-65d46bd0af04, ephemeral_id -> 7203285e-85aa-46f8-b9f1-ca88ef568144, type -> winlogbeat)",The Group Policy Client se